# Writing MAP-EM with SIRF
This notebook provides a start for writing MAP-EM yourself. It gives you basic lines for setting up a simulation, a simplified implementation of MAP-EM, and a few lines to help plot results.

We will use the algorithm described in

Guobao Wang and Jinyi Qi,  
Penalized Likelihood PET Image Reconstruction using Patch-based Edge-preserving Regularization  
IEEE Trans Med Imaging. 2012 Dec; 31(12): 2194–2204.   
[doi:  10.1109/TMI.2012.2211378](https://dx.doi.org/10.1109%2FTMI.2012.2211378)

However, we will not use patches here, but just a simple quadratic prior. Also, for simplicity/speed we will just consider a 1D neighbourhood (horizontal lines)

We will occasionally use the notation of the paper (but not consistently, mainly to avoid conflicts between a coordinate *x* and the image *x*).

You are strongly advised to complete (at least) the first half of the ML_reconstruction exercise before starting with this one. For instance, the simulation and plotting code here is taken directly from the `ML_reconstruction` code, but with fewer comments.

Author: Kris Thielemans  
First version: 19th of May 2018  
Second version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2017 - 2018, 2021 University College London.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging.
(http://www.synerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Brief context on the algorithm
For the Maximum Likelihood problem, EM is a popular choice to derive an optimisation algorithm. For Maximum likelihood, with a suitable choice of "complete data", the "E" step constructs a series of decoupled (i.e. 1D) problems, one for every voxel, which are then easy to solve in the "M" step. However, the penalty term generally makes this a highly coupled set of equations.

In 1995, De Pierro proposed a modified EM algorithm based on the *optimisation transfer* principle, where at each iteration, the function that needs to be optimised is replaced by an "easier" one, often called a *surrogate*. Under certain conditions, such as that the surrogate lies "above" the original function (for minimisation), repeating this process will converge to the true optimum.  The "De Pierro trick" consists in finding a separable surrogate for the penalty term. Therefore, this modified algorithm again solves a system of decoupled 1D equations. Here's the paper (which is unfortunately not so easy to read):

De Pierro, A. R. ‘A Modified Expectation Maximization Algorithm for Penalized Likelihood Estimation in Emission Tomography’. IEEE Transactions on Medical Imaging 14, no. 1 (March 1995): 132–37. https://doi.org/10.1109/42.370409.

The resulting algorithm is often called "De Pierro's MAP-EM" (although not everyone agrees on this terminology, of course).

For a quadratic penalty, the penalty surrogate is also quadratic, and the 1D problems become simple quadratic equations, with an analytic solution. For other penalties, a numerical solution needs to be found. See eq. 43 in the paper, and eq. 60 for the log-cosh prior.

In 2012, Wang&Qi took the surrogate idea one step further and constructed a quadratic surrogate for a large series of (differentiable) penalties, see eq. 28 in the Wang&Qi paper. Of course,for a quadratic prior, the Wang&Qi algorithm is the same as De Pierro's. The notation in the Wang&Qi paper is possibly a bit clearer than De Pierro's, therefore we refer to former below.

(The quadratic surrogate idea was around earlier. See for instance the Separable Parabolic Surrogate paper by Erdogan and Fessler).

## What about subsets?
While the MAP-EM algorithm converges to the true optimum, its update formula uses all of the data (as it reduces to MLEM without penalty). Therefore, computational cost is still high. It is unfortunately not so easy to construct a subset algoqrithm based on the optimisation transfer, so most people will just use the MAP-EM update in an ordered sequence of subsets, which could be called OS-MAP-EM. Note however that this is *not* a convergent algorithm, just like OSEM is not, which could lead to surprises. Nevertheless, this is what we are going to do here.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'MAPEM')

#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
from numba import jit
#import scipy
#from scipy import optimize
import sirf.STIR as pet
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path

In [ ]:
# define the directory with input files for this notebook
data_path = os.path.join(examples_data_path('PET'), 'thorax_single_slice')

# plotting settings
plt.ion() # interactive 'on' such that plots appear during loops

In [ ]:
# set-up redirection of STIR messages to files
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')

In [ ]:
#%% some handy function definitions
def imshow(image, limits, title='', cmap='viridis'):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap=plt.imshow(image, cmap=cmap)
    if len(limits)==0:
        limits=[image.min(),image.max()]

    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off');
    return bitmap

def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV"""
    filter = pet.TruncateToCylinderProcessor()
    filter.apply(image)

# progressbar version of `range`
from tqdm.auto import trange

# Create some sample data

In [ ]:
#%% Read in image
image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
image_array=image.as_array()
cmax = image_array.max()*.6

In [ ]:
#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
am.set_up(template, image)

In [ ]:
#%% simulate some data using forward projection
acquired_data=am.forward(image)

# create OSEM reconstructor

In [ ]:
#%% create objective function
obj_fun = pet.make_Poisson_loglikelihood(acquired_data)
obj_fun.set_acquisition_model(am)

In [ ]:
#%% create OSEM reconstructor
OSEM_reconstructor = pet.OSMAPOSLReconstructor()
OSEM_reconstructor.set_objective_function(obj_fun)
OSEM_reconstructor.set_num_subsets(1)
num_subiters=10
OSEM_reconstructor.set_num_subiterations(num_subiters)

In [ ]:
#%%  create initial image
init_image=image.get_uniform_copy(cmax/4)
make_cylindrical_FOV(init_image)

In [ ]:
#%% initialise
OSEM_reconstructor.set_up(init_image)

# Implement MAP-EM!
Actually, we will implement MAP-OSEM as that's easy to do.

The lines below (almost) implement MAP-OSEM with a prior which just smooths along the horizontal direction.

Please note: this code was written to be simple. It is not terribly safe, very slow, and doesn't use best programming practices (it uses global variables, has no docstrings, no testing of validate of input, etc).

## Define weights as an array
Wang&Qi define their algorithm for non-local weights, which means they use an algorithm to define the weighted smoothing of voxel $j$ from voxel $k$, $w_{jk}$. We will just implement a simpler local smoothing, which means weights are defined only for neighbours. This is equivalent of convolving with a kernel.

The code further below assumes that we have a one-dimensional kernel of length 3 - the weights of which will correspond to smoothing in the previous voxel in the $x$ direction, smoothing with the current voxel ($i=j$), and smoothing with the voxel in the next $x$ direction. Let's just define a simple uniform kernel. We set the $i=j$ voxel to be zero to avoid self-averaging the voxel (see the definition of $x^{reg}$ below.


In [ ]:
w=numpy.array([1.,0.,1.])
# normalise to have sum 1
w/=w.sum()

## define a function that computes $x_{reg}$
For a simple quadratic prior (with normalised weights), $x_{reg}$ (eq. 38) becomes

$x^{reg}_j={1\over 2}\sum_{k\in N_j} w_{jk}(x_j+x_k)$

We will use the "Just In Time" (jit) compilation feature of `numba` here to speed-up the following function.

In [ ]:
@jit
def compute_xreg(image_array):
    sizes=image_array.shape
    image_reg= image_array*0 # make a copy first. Will then change values
    for z in range(0,sizes[0]):
        for y in range(0,sizes[1]):
            for x in range(1,sizes[2]-1): # ignore first and last pixel for simplicity
                for dx in (-1,0,1):
                    image_reg[z,y,x] += w[dx+1]/2*(image_array[z,y,x]+image_array[z,y,x+dx])

    return image_reg

## define a function that computes the MAP-EM update, given $x_{EM}$ and $x_{reg}$

$x^{new}={2 x^{EM}  \over \sqrt{(1-\beta x^{reg})^2 + 4 \beta x^{EM}} + (1-\beta x^{reg})}$

We add an extra $\epsilon=10^{-6}$ to the denominator to avoid division by zero. Note that this isn't mentioned in the original paper, but definitely becomes necessary when you work with subsets.

In [ ]:
@jit
def compute_MAPEM_update(xEM,xreg, beta):
    return 2*xEM/(numpy.sqrt((1 - beta*xreg)**2 + 4*beta*xEM) + (1 - beta*xreg) + 1e-6)

## Define some functions for testing

In [ ]:
#%% define a function for plotting images and the updates
# This is the same function as in `ML_reconstruction`
def plot_progress(all_images, title, subiterations=None, slice_show=0):
    if not subiterations:
        num_subiters = all_images[0].shape[0] - 1
        subiterations = range(1, num_subiters + 1)
    num_rows = len(all_images)
    for k in subiterations:
        plt.figure()
        for r in range(num_rows):
            plt.subplot(num_rows,2,2*r+1)
            imshow(all_images[r][k,slice_show,:,:], [0,cmax], '%s at %d' % (title[r],  k))
            plt.subplot(num_rows,2,2*r+2)
            imshow(all_images[r][k,slice_show,:,:]-all_images[r][k-1,slice_show,:,:],[-cmax*.1,cmax*.1], 'update', cmap='seismic')
        plt.show()

## Finally, let's implement MAP-OSEM!
We have an existing EMML SIRF reconstruction that does the hard work, i.e. compute $x_{EM}$, so let's use that!

Note that if we wouldn't have added a small "epsilon" above, you would get warning messages when you execute this, and the images will not be good. Try it!

In [ ]:
#%% do a loop, saving images as we go along
num_subiters = 32
beta = .1
current_image = init_image.clone()
all_images = numpy.ndarray(shape=(num_subiters+1,) + current_image.as_array().shape )
all_images[0,:,:,:] =  current_image.as_array()
for k in trange(1, num_subiters+1):
    image_reg= compute_xreg(current_image.as_array()) # compute xreg
    OSEM_reconstructor.update(current_image) # compute EM update
    image_EM=current_image.as_array() # get xEM as a numpy array
    updated = compute_MAPEM_update(image_EM, image_reg, beta) # compute new update
    current_image.fill(updated) # store for next iteration
    all_images[k,:,:,:] =  updated # save for plotting later on

In [ ]:
#%% display
slice_show = 0
imshow(current_image.as_array()[slice_show,:,:],[])

In [ ]:
#%% now check how to updates progressed
subiterations = [1,2,4,8,16,32]
plot_progress([all_images], ['MAP-OSEM'], subiterations, slice_show)

## What now?
We list some example ideas for extending this exercise. This will all be more meaningful if you first add some noise to the data of course.
### compare with MLEM/OSEM
### experiment with setting beta
what happens if you increase/decreate beta?

### generalise the `xreg` code
generalise it to 2D (or even 3D)
### check what subsets do
increase the number of subsets and see if the algorithm breaks down, or will it do a limit cycle like OSEM?
### compare with another algorithm solving the same penalised reconstruction
`sirf.STIR` comes at present with 2 other algorithms for MAP reconstruction:
- Green's One Step Late algorithm (`OSMAPOSLReconstructor`) was one of the first MAP-EM algorithms that solved the problem of the EM by plugging in a previous estimate of the image in the penalty term before doing the "M" step (hence the name). Unfortunately, it only converges when the initial image is "close enough" to the converged image. In practice, it means that it diverges for large penalty strength. It is illustrated in the following SIRF examples:
    - [osl_reconstruction.py](https://github.com/SyneRBI/SIRF/blob/v3.1.0/examples/Python/PET/osl_reconstruction.py) shows how to run it with a `sirf.STIR` prior
    - [user_osmaposl.py](https://github.com/SyneRBI/SIRF/blob/v3.1.0/examples/Python/PET/user_osmaposl.py) provides a basic implementation of the OSL algorithm.
- The relaxed version of the Ordered Subsets Separable Paraboloidal Surrogate (OS-SPS with "precomputed denominator") from Ahn and Fessler (`OSSPSReconstructor`). This is a convergent subset algorithm for any prior that allows constructing a separable quadratic surrogate (see above). It is illustrated in [ossps_reconstruction.py](https://github.com/SyneRBI/SIRF/blob/v3.1.0/examples/Python/PET/ossps_reconstruction.py). SIRF 3.1 only provides the STIR wrapper, and does not yet include a SIRF implementation yet. Therefore it can currently only be used with `sirf.STIR` penalties.

Of course, given the objective function, it is possible to use other optimisation algorithms, such as those provided in SciPy and CIL. Remember that you will likely need to use a constrained optimisation algorithm, taking the positivity into account. (Even when doing that, some generic algorithm will fail if there is no background term in the acquisition model, as the likelihood can become ill-defined (i.e. infinity) due to the presence of the logarithm)
### use the log-cosh  penalty
You can do that in 2 ways: using the original De Pierro formulation (i.e. a numerical optimisation) or the Wang&Qi version (i.e. use their formulas to find a quadratic surrogate for the penalty). This would lead to an interesting investigation into whether using quadratic surrogates with penalties that strongly promote edge preservation is a good strategy, or does it lead to slow convergence?